# **Document Search**

In [ ]:
doc_name = "sample.pdf"
doc_content = "This is a sample text used for testing purpose"

# ***Part 1 : Generating bloom filter, Storing in MySQL, Making transaction to smart contract (By Data Owner)***



In [ ]:
# hash functions 

def h1(s, filterSize) :

    hash = 0;
    for i in range(len(s)) :
        hash = hash + ord(s[i])
        
    hash = hash % filterSize 
    return hash

def h2(s, filterSize) :

    hash = 1
    for i in range(len(s)) :
        hash = hash + ord(s[i]) * (19**i)

    hash = hash % filterSize   
    return hash


def h3(s, filterSize) :

    hash = 7
    for i in range(len(s)) :
        hash = (hash * 31 + ord(s[i]))
       
  
    hash = hash % filterSize 
    return hash

wordList = doc_content.split()
print(wordList)

import math

n = len(doc_content) - doc_content.count(" ")  # max possible distinct trigrams
prob = 0.1  # desired false positive probability
m = math.ceil(-(n * math.log(prob)) / (math.log(2)*math.log(2)))  # size of bloom filter
k = 3   # number of hash funtions 

bloom_filter = [0]*m

def get_trigrams(s) :
    n = len(s)
    s = s + s + s;
    res = []
    for i in range(n) :
        res.append(s[i:i+3])

    return res

def checkIfPresent(trigram, bloom_filter) :
    
    m = len(bloom_filter)
    hash1 = h1(trigram, m)
    hash2 = h2(trigram, m)
    hash3 = h3(trigram, m) 

    if bloom_filter[hash1] == 1 and bloom_filter[hash2] == 1 and bloom_filter[hash3] == 1 :
        return 1
    
    return 0

def generate_bloom_filter(wordList, bloom_filter) :

    m = len(bloom_filter)

    for word in wordList :
        trigramList = get_trigrams(word)

        for trigram in trigramList :

            var = checkIfPresent(trigram, bloom_filter)

            if var == 1 :
              continue

            hash1 = h1(trigram, m)
            hash2 = h2(trigram, m)
            hash3 = h3(trigram, m) 

            bloom_filter[hash1] = 1 
            bloom_filter[hash2] = 1
            bloom_filter[hash3] = 1

['This', 'is', 'a', 'sample', 'text', 'used', 'for', 'testing', 'purpose']


In [ ]:
generate_bloom_filter(wordList, bloom_filter)

In [ ]:
# print bloom_filter contents

print("Size of bloom filter :", len(bloom_filter))

print("Contents of bloom filter :")
for i in range(len(bloom_filter)) :
    print(str(bloom_filter[i]) + ",", end='')

Size of bloom filter : 183
Contents of bloom filter :
1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,1,

In [ ]:
!pip install cryptography

# Convert to string in ascii from numeric representation of triplets
def convert(a) :
    l = len(a)
    s = ""
    
    idx = 0
    while idx < l :
        temp = a[idx:idx+3]
        s = s + chr(int(temp))
        idx = idx + 3
        
    return s

# Creating a random salt of length n
import string
import random

def get_random_salt(n) :
  
    salt=""
    for i in range(n):
        salt=salt+random.choice(string.ascii_letters)

    return salt

# Get a random encryption key
from cryptography.fernet import Fernet

def get_random_encryptkey() :

    encrypt_key = Fernet.generate_key()
    # fernet = Fernet(key)
    return encrypt_key

# Take XOR of string a and string b
def XOR1(a, b) :
    res = ""
    i = 0
    j = 0
    
    while (i<len(a) and j<len(b)) :
        
        temp = str(ord(a[i]) ^ ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        j = j + 1
    
    while i<len(a) :
        
        temp = str(ord(a[i])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        
    while j<len(b) :
        
        temp = str(ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        j = j + 1
        
    return res   


In [ ]:
pip install mysql-connector-python-rf

In [ ]:
# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

 * Starting MySQL database server mysqld
   ...done.
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * mysql+pymysql://root:***@/


,Host,User,authentication_string
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B
1,localhost,mysql.session,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
2,localhost,mysql.sys,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
3,localhost,debian-sys-maint,*4B8859D18B4E30933381F26939A7AB5C094F00D2


In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

mycursor = mydb.cursor()

In [ ]:
mycursor.execute("DROP DATABASE btp")

In [ ]:
mycursor.execute("CREATE DATABASE btp")

In [ ]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('information_schema',)
('btp',)
('mysql',)
('performance_schema',)
('sys',)


In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database='btp'
)

mycursor = mydb.cursor()

In [ ]:
mycursor.execute("DROP TABLE Documents")

In [ ]:
mycursor.execute("CREATE TABLE Documents ( doc_name VARCHAR(511), doc_content VARCHAR(64535) )")

In [ ]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

('Documents',)


In [ ]:
def store_data(doc_name, doc_content) : 

    # insert data query 
    insert_query = "INSERT INTO Documents (doc_name, doc_content) VALUES (%s, %s)"
   
    encrypt_key = get_random_encryptkey();
    fernet = Fernet(encrypt_key)

    encrypted_name = fernet.encrypt(doc_name.encode('utf-8'))
    salt = get_random_salt(len(encrypted_name))
    encrypted_name_with_salt = XOR1(str(encrypted_name),salt)

    encrypted_content = fernet.encrypt(doc_content.encode('utf-8'))
    encrypted_content_with_salt = XOR1(str(encrypted_content),salt)

    encrypted_data = (encrypted_name_with_salt, encrypted_content_with_salt)
    mycursor.execute(insert_query, encrypted_data)
    mydb.commit()

    
    print("Document Name : ", doc_name)

    print("Encryption Key :", encrypt_key)

    print("Salt : ", salt)
    
    print("Encryted Document Name : ", encrypted_name)

In [ ]:
store_data(doc_name, doc_content)

In [ ]:
# print bloom_filter contents

print("Size of bloom filter :", len(bloom_filter))

print("Contents of bloom filter :")
for i in range(len(bloom_filter)) :
    print(str(bloom_filter[i]) + ",", end='')

Size of bloom filter : 183
Contents of bloom filter :
1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,1,

In [ ]:
mycursor.execute("SELECT * from Documents")
for x in mycursor:
  print(x)

('010106014010043059049054053005018115010043054018015050006040061011093032046022036058041062050002046057103002052062043095044047059030018050025004014063059004036026042047017092038050039010041053058124054007045041001027007028002037000006003029023032081024063004011004021028039015019069031008006062021061061061039', '010106014010043059049054053005018115010043005018006024044021021054023024046085001095041020017004118022045007040027000080054006038032104023014057060058087025009011044052027092044009003083062006070015058063028029043039043023073124021029031107101058084028102032006018000056016082069049002008052063042032107108086103066054112111073050077077106071077090105055068056084102118076102105082070110085099084116054067055053045090069090066039')


# **Part 2 : Query Doc Content from MySQL after getting salt from smart contract (By Data User)**

In [ ]:

# Get from result of store_data 
key1 = b'W3bCm68BGlUGnTuxkZJvLVBlY0FUn3NhqXkivTC1QB4='
EncDoc_name = "b'gAAAAABhp7aIWSGHAEIx3pJqFRxHKGaN2dlOF9X_pUHuxEmqTprXEZg5SClmdWO4otEyiNtMrmeTDEEu2liSCbxvOkf2gPuMrQ=='"
salt = "hMiKjzpwwmbDkbaAHzGmtsnPdgbhQvyEOwUfXqmftpKKZGaAcNotVBouviuqKgMbuHYshPhUsQpHeRGXRUctVWHfmjhduwxXssgl" 

fernet1=Fernet(key1)

Idd=(XOR1(salt,EncDoc_name), )


query = "SELECT doc_content from Documents where doc_name= %s"
mycursor.execute(query,Idd)
res=""
for x in mycursor:
  #print(x)
  res=x

Encrypted_res=XOR1(convert(res[0]),salt)
#print(Encrypted_res)
#print(type(res[0]))

temp=convert(Encrypted_res)
print(temp)
s1=temp[1:len(temp)]
decMessage = fernet1.decrypt(s1.encode('utf-8')).decode()

print("\n\nDecrypted string: ", decMessage)

b'gAAAAABhp7aIdSNbkxaEyHJ2c7xbhA9axapjm6Bvmk2Pox_t8m_ICAm5YxH4sd3GcLtMCrXF94pOX37o7h0wNtmRx60FzPGLMLklVgB6poI2MMjGMZi7D8TfvLfiRFnUcTt6C75-ZEZB'


Decrypted string:  This is a sample text used for testing purpose
